In [12]:
import pandas as pd

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("varsharam/walmart-sales-dataset-of-45stores")

print("Path to dataset files:", path)

Path to dataset files: /Users/guilhermemartins/.cache/kagglehub/datasets/varsharam/walmart-sales-dataset-of-45stores/versions/1


In [14]:
import os
# Carregando um dos arquivos CSV com pandas
csv_path = os.path.join(path, "walmart-sales-dataset-of-45stores.csv")  # Exemplo de nome; ajuste conforme o print acima
df = pd.read_csv(csv_path)
print("Path to dataset files:", path)

Path to dataset files: /Users/guilhermemartins/.cache/kagglehub/datasets/varsharam/walmart-sales-dataset-of-45stores/versions/1


In [15]:
import findspark  # This helps us find and use Apache Spark
findspark.init()  # Initialize findspark to locate Spark
from pyspark.sql import SparkSession  
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DateType
import pandas as pd  

# Initialize a Spark Session
spark = SparkSession \
    .builder \
    .appName("Wallmart Sales Analysis") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.debug.maxToStringFields", 100) \
    .getOrCreate()

# Check if the Spark Session is active
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession is active and ready to use.")
else:
    print("SparkSession is not active. Please create a SparkSession.")

25/05/19 11:32:24 WARN Utils: Your hostname, Guilhermes-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.15.6 instead (on interface en0)
25/05/19 11:32:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/19 11:32:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession is active and ready to use.


In [16]:
# Converter DataFrame Pandas para Spark
df_spark = spark.createDataFrame(df)

In [17]:
df_spark.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
|    1|05-02-2010|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|
|    1|12-02-2010|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|
|    1|19-02-2010|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|
|    1|26-02-2010|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|
|    1|05-03-2010|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
only showing top 5 rows



['Weekly_Sales', 'features3']

25/05/17 04:05:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 941699 ms exceeds timeout 120000 ms
25/05/17 04:05:47 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/17 04:05:54 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [9]:
from pyspark.sql.functions import to_date

# Formato padrão: "yyyy-MM-dd" (ajuste conforme seus dados)
df_spark = df_spark.withColumn("Date", to_date("Date", "dd-MM-yyyy"))

In [10]:
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

In [11]:
 # Datas dos feriados (ajuste o formato conforme seus dados)
feriados = {
    "Super_Bowl": ["12-02-2010", "11-02-2011", "10-02-2012"],  # Formato DD-MM-YYYY
    "Labour_Day": ["10-09-2010", "09-09-2011", "07-09-2012"],
    "Thanksgiving": ["26-11-2010", "25-11-2011", "23-11-2012"],
    "Christmas": ["31-12-2010", "30-12-2011", "28-12-2012"]
}

In [12]:
# Spark
from pyspark.sql import functions as F
# Criar colunas para cada feriado
for feriado, datas in feriados.items():
    df_spark = df_spark.withColumn(
        feriado,
        F.when(
            F.col("Date").isin([F.to_date(F.lit(data), "dd-MM-yyyy") for data in datas]),
            1
        ).otherwise(0)
    )

In [13]:
# Pandas
for feriado, datas in feriados.items():
    datas_dt = pd.to_datetime(datas, format='%d-%m-%Y')  # converte lista para datetime
    df[feriado] = df['Date'].isin(datas_dt).astype(int)

In [14]:
df_spark.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|Super_Bowl|Labour_Day|Thanksgiving|Christmas|
+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+
|    1|2010-02-05|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|         0|         0|           0|        0|
|    1|2010-02-12|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|         1|         0|           0|        0|
|    1|2010-02-19|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|         0|         0|           0|        0|
|    1|2010-02-26|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|         0|         0|           0|        0|
|    1|2010-03-05|  1554806.68|   

In [18]:
df_spark.createOrReplaceTempView("vendas")

In [30]:
spark.sql("Select Weekly_Sales,Temperature FROM vendas ORDER BY Temperature DESC").show()

+------------+-----------+
|Weekly_Sales|Temperature|
+------------+-----------+
|   280937.84|     100.14|
|   297753.49|     100.07|
|   237095.82|      99.66|
|   912403.67|      99.22|
|   328336.85|      99.22|
|  1205884.98|      99.22|
|   239198.36|       99.2|
|   226702.36|      98.43|
|   244338.31|      98.15|
|   267495.76|      97.66|
|   287033.64|       97.6|
|   338277.71|      97.18|
|   916402.76|      97.18|
|  1225336.41|      97.18|
|   238915.05|      97.17|
|   252734.31|      97.04|
|   242456.39|      96.93|
|   270097.76|      96.79|
|   230519.49|      96.46|
|   262717.71|      96.44|
+------------+-----------+
only showing top 20 rows



In [26]:
spark.sql("SELECT Date, Store FROM vendas WHERE Weekly_Sales > 1643690.9").show(50)


+----------+-----+
|      Date|Store|
+----------+-----+
|2010-11-26|    1|
|2010-12-10|    1|
|2010-12-17|    1|
|2010-12-24|    1|
|2011-02-11|    1|
|2011-02-18|    1|
|2011-11-04|    1|
|2011-11-25|    1|
|2011-12-09|    1|
|2011-12-16|    1|
|2011-12-23|    1|
|2012-02-10|    1|
|2012-02-17|    1|
|2012-03-02|    1|
|2012-03-09|    1|
|2012-03-16|    1|
|2012-03-30|    1|
|2012-04-06|    1|
|2012-05-04|    1|
|2012-06-08|    1|
|2012-07-06|    1|
|2012-09-07|    1|
|2012-10-05|    1|
|2010-02-05|    2|
|2010-02-12|    2|
|2010-02-19|    2|
|2010-02-26|    2|
|2010-03-05|    2|
|2010-03-12|    2|
|2010-03-19|    2|
|2010-03-26|    2|
|2010-04-02|    2|
|2010-04-09|    2|
|2010-04-16|    2|
|2010-04-23|    2|
|2010-04-30|    2|
|2010-05-07|    2|
|2010-05-14|    2|
|2010-05-21|    2|
|2010-05-28|    2|
|2010-06-04|    2|
|2010-06-11|    2|
|2010-06-18|    2|
|2010-06-25|    2|
|2010-07-02|    2|
|2010-07-09|    2|
|2010-07-16|    2|
|2010-07-23|    2|
|2010-07-30|    2|
|2010-08-06|

In [36]:
spark.sql("SELECT Store, Weekly_Sales, Super_Bowl FROM vendas WHERE Super_Bowl = 1 ORDER BY Weekly_Sales DESC").show()

+-----+------------+----------+
|Store|Weekly_Sales|Super_Bowl|
+-----+------------+----------+
|   20|  2462978.28|         1|
|    4|  2374660.64|         1|
|   10|   2218595.8|         1|
|   20|  2211388.14|         1|
|    4|  2188307.39|         1|
|    4|  2187847.29|         1|
|   10|  2176028.52|         1|
|    2|  2168041.61|         1|
|    2|   2137809.5|         1|
|   10|  2115408.31|         1|
|   20|   2109107.9|         1|
|    2|  2103322.68|         1|
|   14|  2077256.24|         1|
|   13|  2069284.57|         1|
|   13|  2030933.46|         1|
|   14|  1980405.03|         1|
|   13|   1944438.9|         1|
|    1|  1802477.43|         1|
|   27|  1745362.72|         1|
|   14|  1704218.84|         1|
+-----+------------+----------+
only showing top 20 rows



In [35]:
spark.sql("SELECT Store, Weekly_Sales, Christmas FROM vendas WHERE Christmas= 1 ORDER BY Weekly_Sales DESC").show()

+-----+------------+---------+
|Store|Weekly_Sales|Christmas|
+-----+------------+---------+
|   20|   2043245.0|        1|
|    4|  2007105.86|        1|
|   13|  1969056.91|        1|
|   10|  1930690.37|        1|
|   14|  1914148.89|        1|
|    2|  1874226.52|        1|
|   20|  1799737.79|        1|
|    4|  1794868.74|        1|
|    2|  1750434.55|        1|
|   10|  1707298.14|        1|
|   13|   1675292.0|        1|
|   27|   1650604.6|        1|
|   14|  1623716.46|        1|
|    6|  1598080.52|        1|
|   39|  1537139.56|        1|
|    1|  1497462.72|        1|
|    6|  1464050.02|        1|
|   27|   1440963.0|        1|
|   19|  1405168.06|        1|
|    1|  1367320.01|        1|
+-----+------------+---------+
only showing top 20 rows



In [37]:
spark.sql("SELECT MAX(Weekly_Sales) FROM vendas").show()

+-----------------+
|max(Weekly_Sales)|
+-----------------+
|       3818686.45|
+-----------------+



In [53]:
numerical_features = df.select_dtypes(include=['number']).columns.tolist()
numerical_features = [col for col in numerical_features if col != 'Weekly_Sales']

for feature in numerical_features:
    correlation = df[feature].corr(df['Weekly_Sales'])
    print(f"Correlação entre {feature} e Weekly_Sales: {correlation}")


Correlação entre Store e Weekly_Sales: -0.33533201479281577
Correlação entre Holiday_Flag e Weekly_Sales: 0.036890968010414464
Correlação entre Temperature e Weekly_Sales: -0.06381001317946962
Correlação entre Fuel_Price e Weekly_Sales: 0.009463786314475135
Correlação entre CPI e Weekly_Sales: -0.0726341620401762
Correlação entre Unemployment e Weekly_Sales: -0.1061760896579542
Correlação entre Super_Bowl e Weekly_Sales: 0.008343092798194991
Correlação entre Labour_Day e Weekly_Sales: -0.001177046659554076
Correlação entre Thanksgiving e Weekly_Sales: 0.08954877546732833
Correlação entre Christmas e Weekly_Sales: -0.018177795782735418


In [54]:
df_spark.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|Super_Bowl|Labour_Day|Thanksgiving|Christmas|
+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+
|    1|2010-02-05|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|         0|         0|           0|        0|
|    1|2010-02-12|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|         1|         0|           0|        0|
|    1|2010-02-19|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|         0|         0|           0|        0|
|    1|2010-02-26|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|         0|         0|           0|        0|
|    1|2010-03-05|  1554806.68|   

In [13]:
from pyspark.sql.functions import to_date

#
df_spark = df_spark.withColumn("date", to_date("Date", "dd-MM-yyyy"))

In [55]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek
# Extrair features temporais
df_spark = df_spark.withColumn("year", year("date")) \
                   .withColumn("month", month("date")) \
                   .withColumn("day", dayofmonth("date")) \
                   .withColumn("day_of_week", dayofweek("date"))

In [56]:
df_spark.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+----+-----+---+-----------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|Super_Bowl|Labour_Day|Thanksgiving|Christmas|year|month|day|day_of_week|
+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+----+-----+---+-----------+
|    1|2010-02-05|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|         0|         0|           0|        0|2010|    2|  5|          6|
|    1|2010-02-12|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|         1|         0|           0|        0|2010|    2| 12|          6|
|    1|2010-02-19|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|         0|         0|           0|        0|2010|    2| 19|          6|
|    1|201

In [57]:
df_spark.columns

['Store',
 'Date',
 'Weekly_Sales',
 'Holiday_Flag',
 'Temperature',
 'Fuel_Price',
 'CPI',
 'Unemployment',
 'Super_Bowl',
 'Labour_Day',
 'Thanksgiving',
 'Christmas',
 'year',
 'month',
 'day',
 'day_of_week']

In [58]:
from pyspark.sql.functions import col

# Lista de colunas a verificar
columns_to_check = ['Store', 'date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 
                    'CPI', 'Unemployment', 'Super_Bowl', 'Labour_Day', 'Thanksgiving', 
                    'Christmas', 'year', 'month', 'day', 'day_of_week']

# Verificar e contar valores nulos por coluna
null_counts = {col_name: df_spark.filter(col(col_name).isNull()).count() for col_name in columns_to_check}

# Exibir os resultados
for col_name, count in null_counts.items():
    print(f"Coluna {col_name}: {count} valores nulos")

Coluna Store: 0 valores nulos
Coluna date: 0 valores nulos
Coluna Weekly_Sales: 0 valores nulos
Coluna Holiday_Flag: 0 valores nulos
Coluna Temperature: 0 valores nulos
Coluna Fuel_Price: 0 valores nulos
Coluna CPI: 0 valores nulos
Coluna Unemployment: 0 valores nulos
Coluna Super_Bowl: 0 valores nulos
Coluna Labour_Day: 0 valores nulos
Coluna Thanksgiving: 0 valores nulos
Coluna Christmas: 0 valores nulos
Coluna year: 0 valores nulos
Coluna month: 0 valores nulos
Coluna day: 0 valores nulos
Coluna day_of_week: 0 valores nulos


In [59]:
input_cols = ['Store',
 'Holiday_Flag',
 'Temperature',
 'CPI',
 'Unemployment',
 'Thanksgiving',
 'Christmas',
 'year',
 'month',
 'day',
 'day_of_week']

In [60]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [62]:
from pyspark.ml.feature import VectorAssembler

# Criar o VectorAssembler
colunas_irrelevantes = ['Fuel_Price', 'Super_Bowl', 'Labour_Day', 'Date'] 
feature_cols = [col for col in df_spark.columns if col not in colunas_irrelevantes and col != 'Weekly_Sales']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features3")
df_spark = assembler.transform(df_spark)



In [63]:
df_spark.printSchema()

root
 |-- Store: long (nullable = true)
 |-- Date: date (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- Holiday_Flag: long (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- CPI: double (nullable = true)
 |-- Unemployment: double (nullable = true)
 |-- Super_Bowl: integer (nullable = false)
 |-- Labour_Day: integer (nullable = false)
 |-- Thanksgiving: integer (nullable = false)
 |-- Christmas: integer (nullable = false)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- features3: vector (nullable = true)



In [64]:
df_spark.show(5)

+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+----+-----+---+-----------+--------------------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|Super_Bowl|Labour_Day|Thanksgiving|Christmas|year|month|day|day_of_week|           features3|
+-----+----------+------------+------------+-----------+----------+-----------+------------+----------+----------+------------+---------+----+-----+---+-----------+--------------------+
|    1|2010-02-05|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|         0|         0|           0|        0|2010|    2|  5|          6|[1.0,0.0,42.31,21...|
|    1|2010-02-12|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|         1|         0|           0|        0|2010|    2| 12|          6|[1.0,1.0,38.51,21...|
|    1|2010-02-19|  1611968.17|           0|      39.93|     2.514|211

In [65]:
# Selecionar colunas relevantes para o modelo
df_spark = df_spark.select("Weekly_Sales", "features3")
df_spark.show(5, truncate=False)

+------------+-------------------------------------------------------------+
|Weekly_Sales|features3                                                    |
+------------+-------------------------------------------------------------+
|1643690.9   |[1.0,0.0,42.31,211.0963582,8.106,0.0,0.0,2010.0,2.0,5.0,6.0] |
|1641957.44  |[1.0,1.0,38.51,211.2421698,8.106,0.0,0.0,2010.0,2.0,12.0,6.0]|
|1611968.17  |[1.0,0.0,39.93,211.2891429,8.106,0.0,0.0,2010.0,2.0,19.0,6.0]|
|1409727.59  |[1.0,0.0,46.63,211.3196429,8.106,0.0,0.0,2010.0,2.0,26.0,6.0]|
|1554806.68  |[1.0,0.0,46.5,211.3501429,8.106,0.0,0.0,2010.0,3.0,5.0,6.0]  |
+------------+-------------------------------------------------------------+
only showing top 5 rows



In [100]:
# Checando novamente valores nulos
from pyspark.sql.functions import col
null_counts = {col_name: df_spark.filter(col(col_name).isNull()).count() for col_name in input_cols}
for col_name, count in null_counts.items():
    print(f"Coluna {col_name}: {count} valores nulos")

Coluna Store: 0 valores nulos
Coluna Holiday_Flag: 0 valores nulos
Coluna Temperature: 0 valores nulos
Coluna CPI: 0 valores nulos
Coluna Unemployment: 0 valores nulos
Coluna Thanksgiving: 0 valores nulos
Coluna Christmas: 0 valores nulos
Coluna year: 0 valores nulos
Coluna month: 0 valores nulos
Coluna day: 0 valores nulos
Coluna day_of_week: 0 valores nulos


In [69]:
# Split the data
train_data_spark, test_data_spark = df_spark.randomSplit([0.8, 0.2], seed=42)

In [66]:
df_spark.show(5)

+------------+--------------------+
|Weekly_Sales|           features3|
+------------+--------------------+
|   1643690.9|[1.0,0.0,42.31,21...|
|  1641957.44|[1.0,1.0,38.51,21...|
|  1611968.17|[1.0,0.0,39.93,21...|
|  1409727.59|[1.0,0.0,46.63,21...|
|  1554806.68|[1.0,0.0,46.5,211...|
+------------+--------------------+
only showing top 5 rows



In [112]:
# Train a Random Forest model
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol="features3", labelCol="Weekly_Sales", numTrees=20, seed=42)
rf_model = rf.fit(train_data_spark)
rf_predictions = rf_model.transform(test_data_spark)
evaluator_rf = RegressionEvaluator(labelCol="Weekly_Sales")
print(f"RF RMSE: {evaluator_rf.evaluate(rf_predictions, {evaluator_rf.metricName: 'rmse'})}")
print(f"RF R2: {evaluator_rf.evaluate(rf_predictions, {evaluator_rf.metricName: 'r2'})}")

RF RMSE: 398472.4917170457
RF R2: 0.4624575598625814


In [113]:
import pandas as pd

importances_rf = rf_model.featureImportances.toArray()

if len(importances_rf) == len(feature_cols):
    df_importance_rf = pd.DataFrame({
        'Feature': feature_cols,
        'Importance_RF': importances_rf
    }).sort_values(by='Importance_RF', ascending=False)
    print("Importância das Features - Random Forest:")
    print(df_importance_rf.head(10))
else:
    print("Erro: Número de features não corresponde ao modelo RF")

Importância das Features - Random Forest:
        Feature  Importance_RF
0         Store       0.583347
4  Unemployment       0.170788
3           CPI       0.142607
8         month       0.041089
2   Temperature       0.028630
9           day       0.015418
5  Thanksgiving       0.008065
7          year       0.004224
1  Holiday_Flag       0.003399
6     Christmas       0.002433


In [ ]:
# Gradient Boost Tree Regressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Modelo
gbt = GBTRegressor(featuresCol="features3", labelCol="Weekly_Sales", maxIter=50)

# Treinar
gbt_model = gbt.fit(train_data_spark)

# Prever e avaliar
gbt_predictions = gbt_model.transform(test_data_spark)

# Métricas
evaluator_gbt = RegressionEvaluator(labelCol="Weekly_Sales")
print(f"GBT RMSE: {evaluator_gbt.evaluate(gbt_predictions, {evaluator_gbt.metricName: 'rmse'})}")
print(f"GBT R2: {evaluator_gbt.evaluate(gbt_predictions, {evaluator_gbt.metricName: 'r2'})}")

25/05/16 20:38:15 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


GBT RMSE: 181167.2578936221
GBT R2: 0.8888842839339648


In [79]:

importances_gbt = gbt_model.featureImportances.toArray()

df_importance_gbt = pd.DataFrame({
    'Feature': feature_cols,
    'Importance_GBT': importances_gbt
}).sort_values(by='Importance_GBT', ascending=False)
print(df_importance_gbt.head(10))

        Feature  Importance_GBT
0         Store        0.425216
3           CPI        0.238000
4  Unemployment        0.184984
2   Temperature        0.042535
9           day        0.035966
8         month        0.033821
7          year        0.021895
5  Thanksgiving        0.012269
6     Christmas        0.003583
1  Holiday_Flag        0.001732


In [85]:
from pyspark.ml.regression import LinearRegression

# Lasso (L1)
lr_lasso = LinearRegression(featuresCol="features3", labelCol="Weekly_Sales", 
                          elasticNetParam=1.0, regParam=0.1)

# Ridge (L2)
lr_ridge = LinearRegression(featuresCol="features3", labelCol="Weekly_Sales", 
                          elasticNetParam=0.0, regParam=0.1)

# Treinar ambos
lasso_model = lr_lasso.fit(train_data_spark)
ridge_model = lr_ridge.fit(train_data_spark)

# Prever e avaliar
lasso_predictions = lasso_model.transform(test_data_spark)
ridge_predictions = ridge_model.transform(test_data_spark)

evaluator_ridge = RegressionEvaluator(labelCol="Weekly_Sales")  # Renomeie se necessário  # Métrica diferente para Lasso

print(f"RIDGE RMSE: {evaluator_ridge.evaluate(ridge_predictions, {evaluator_ridge.metricName: 'rmse'})}")
print(f"RIDGE R2: {evaluator_ridge.evaluate(ridge_predictions, {evaluator_ridge.metricName: 'r2'})}")

evaluator_lasso = RegressionEvaluator(labelCol="Weekly_Sales")
print(f"LASSO RMSE: {evaluator_lasso.evaluate(lasso_predictions, {evaluator_lasso.metricName: 'rmse'})}")
print(f"LASSO R2: {evaluator_lasso.evaluate(lasso_predictions, {evaluator_lasso.metricName: 'r2'})}")

25/05/16 20:49:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


RIDGE RMSE: 497456.4403781198
RIDGE R2: 0.16222734058622656
LASSO RMSE: 497456.4447413191
LASSO R2: 0.16222732588998856


In [91]:
import numpy as np
# Extrair coeficientes (importância absoluta)
lasso_coef = np.abs(lasso_model.coefficients.toArray())

df_importance_lasso = pd.DataFrame({
    'Feature': feature_cols,
    'Importance_Lasso': lasso_coef
}).sort_values(by='Importance_Lasso', ascending=False)

print("\nImportância das Features - Lasso (L1):")
print(df_importance_lasso.head(10))


Importância das Features - Lasso (L1):
        Feature  Importance_Lasso
5  Thanksgiving     269230.122612
6     Christmas     200336.357560
4  Unemployment      21138.046991
0         Store      15363.467987
8         month      14659.157660
3           CPI       2309.231670
2   Temperature       1993.216992
1  Holiday_Flag       1868.649164
7          year       1158.729230
9           day        696.455133


In [92]:
# Extrair coeficientes (importância absoluta)
ridge_coef = np.abs(ridge_model.coefficients.toArray())

df_importance_ridge = pd.DataFrame({
    'Feature': feature_cols,
    'Importance_Ridge': ridge_coef
}).sort_values(by='Importance_Ridge', ascending=False)

print("\nImportância das Features - Ridge (L2):")
print(df_importance_ridge.head(10))


Importância das Features - Ridge (L2):
        Feature  Importance_Ridge
5  Thanksgiving     269232.136713
6     Christmas     200336.609275
4  Unemployment      21138.052075
0         Store      15363.473918
8         month      14659.207857
3           CPI       2309.233259
2   Temperature       1993.228327
1  Holiday_Flag       1869.506152
7          year       1158.914650
9           day        696.469857


In [83]:
# Instale primeiro: pip install xgboost pyspark
from xgboost.spark import SparkXGBRegressor

xgb = SparkXGBRegressor(features_col="features3", label_col="Weekly_Sales",
                       num_workers=2, max_depth=5)

# Treinar
xgb_model = xgb.fit(train_data_spark)

evaluator_xgb = RegressionEvaluator(labelCol="Weekly_Sales")

# Avaliar
xgb_predictions = xgb_model.transform(test_data_spark)
print(f"XGBoost RMSE: {evaluator_xgb.evaluate(xgb_predictions, {evaluator_xgb.metricName: 'rmse'})}")
print(f"XGBoost R2: {evaluator_xgb.evaluate(xgb_predictions, {evaluator_xgb.metricName: 'r2'})}")
# XGBoost RMSE: 86918.09156164837
# XGBoost R2: 0.974423775729908

2025-05-16 20:44:14,744 INFO XGBoost-PySpark: _fit Running xgboost-3.0.1 on 2 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'max_depth': 5, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2025-05-16 20:44:16,396 INFO XGBoost-PySpark: _train_booster Training on CPUs 2]
[20:44:17] Task 0 got rank 0
[20:44:17] Task 1 got rank 1
[20:44:17] [0]	training-rmse:457795.50477
[20:44:17] [1]	training-rmse:388317.92393
[20:44:17] [2]	training-rmse:330389.68210
[20:44:17] [3]	training-rmse:286397.42033
[20:44:17] [4]	training-rmse:264140.66983
[20:44:17] [5]	training-rmse:236015.54882
[20:44:17] [6]	training-rmse:211330.69004
[20:44:17] [7]	training-rmse:202784.30023
[20:44:17] [8]	training-rmse:193762.19478
[20:44:17] [9]	training-rmse:173677.91923
[20:44:17] [10]	training-rmse:167885.63674
[20:44:17] [11]	training-rmse:163466.53493
[20:44:17] [12]	training-rmse:154400.03538
[

XGBoost RMSE: 86781.98764726309


2025-05-16 20:44:20,407 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


XGBoost R2: 0.9745038119739273


In [98]:
# Pegando as importâncias do modelo
importances_xgb = xgb_model.get_booster().get_score(importance_type='weight')

# Criando um DataFrame para visualização
import pandas as pd
feature_importance = pd.DataFrame(list(importances_xgb.items()), columns=['Feature', 'Importance'])
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

print(feature_importance)

  Feature  Importance
0      f0       808.0
2      f2       493.0
3      f3       440.0
9      f9       354.0
4      f4       308.0
8      f8       306.0
1      f1        52.0
7      f7        46.0
5      f5        27.0
6      f6        14.0


In [99]:


importances_xgb_full = []

for i in range(len(feature_cols)):
    feat_name = f'f{i}'  # XGBoost usa nomes como 'f0', 'f1', ...
    importance = importances_xgb.get(feat_name, 0)  # pega importância ou zero se não existir
    importances_xgb_full.append(importance)

print(f"Tamanho do vetor ajustado do XGBoost: {len(importances_xgb_full)}")  # Deve ser igual a len(feature_cols)

Tamanho do vetor ajustado do XGBoost: 11


In [90]:
# Exibir a lista de features em ordem
for idx, feature in enumerate(feature_cols):
    print(f'f{idx}: {feature}')

f0: Store
f1: Holiday_Flag
f2: Temperature
f3: CPI
f4: Unemployment
f5: Thanksgiving
f6: Christmas
f7: year
f8: month
f9: day
f10: day_of_week


TESTES CONSIDERANDO TODAS AS VARIAVEIS

RF RMSE: 415918.99163985276
RF R2: 0.4143561724065379

GBT RMSE: 178833.26130438715
GBT R2: 0.8917288721822565

RIDGE RMSE: 501582.6005249428
RIDGE R2: 0.1482718660863961

LASSO RMSE: 501582.6098245606
LASSO R2: 0.148271834503378

XGBoost RMSE: 86918.09156164837
XGBoost R2: 0.974423775729908


DESCONSIDERANDO FEATURES QUE POSSUEM INFLUENCIA INFERIOR A 0.01.

RF RMSE: 398472.4917170457
RF R2: 0.4624575598625814

GBT RMSE: 181167.2578936221
GBT R2: 0.8888842839339648

RIDGE RMSE: 497456.4403781198
RIDGE R2: 0.16222734058622656

LASSO RMSE: 497456.4447413191
LASSO R2: 0.16222732588998856

XGBoost RMSE: 86781.98764726309
XGBoost R2: 0.9745038119739273

FEATURE IMPORTANCE

Importância das Features - Random Forest:
        Feature  Importance_RF
0         Store       0.583347
4  Unemployment       0.170788
3           CPI       0.142607
8         month       0.041089
2   Temperature       0.028630
9           day       0.015418
5  Thanksgiving       0.008065
7          year       0.004224
1  Holiday_Flag       0.003399
6     Christmas       0.002433

=============================================
Importância das Features - GBT:
        Feature  Importance_GBT
0         Store        0.425216
3           CPI        0.238000
4  Unemployment        0.184984
2   Temperature        0.042535
9           day        0.035966
8         month        0.033821
7          year        0.021895
5  Thanksgiving        0.012269
6     Christmas        0.003583
1  Holiday_Flag        0.001732

=============================================

Importância das Features - Lasso (L1):
        Feature  Importance_Lasso
5  Thanksgiving     269230.122612
6     Christmas     200336.357560
4  Unemployment      21138.046991
0         Store      15363.467987
8         month      14659.157660
3           CPI       2309.231670
2   Temperature       1993.216992
1  Holiday_Flag       1868.649164
7          year       1158.729230
9           day        696.455133

==============================================


Importância das Features - Ridge (L2):
        Feature  Importance_Ridge
5  Thanksgiving     269232.136713
6     Christmas     200336.609275
4  Unemployment      21138.052075
0         Store      15363.473918
8         month      14659.207857
3           CPI       2309.233259
2   Temperature       1993.228327
1  Holiday_Flag       1869.506152
7          year       1158.914650
9           day        696.469857

=============================================
Importância das Features - XGBoost:
          Feature  Importance_XGB
0           Store       808.0
2     Temperature       493.0
3             CPI       440.0
9             day       354.0
4    Unemployment       308.0
8           month       306.0
1    Holiday_Flag        52.0
7            year        46.0
5    Thanksgiving        27.0
6       Christmas        14.0



In [114]:
# Visualizando as importâncias com mais precisão
importances_rf_array = importances_rf.toArray()
importances_gbt_array = importances_gbt.toArray()

# Exibindo com mais casas decimais
for idx, (rf, gbt) in enumerate(zip(importances_rf_array, importances_gbt_array)):
    print(f"Feature: {feature_cols[idx]}, RF Importance: {rf:.10f}, GBT Importance: {gbt:.10f}")

AttributeError: 'numpy.ndarray' object has no attribute 'toArray'

In [93]:
import pandas as pd

# Dados dos modelos
model_metrics = {
    'Model': ['Random Forest', 'Gradient Boosting', 'Ridge', 'Lasso', 'XGBoost'],
    'RMSE': [398472.49, 181167.26, 497456.44, 497456.44, 86781.99],
    'R2': [0.462, 0.888, 0.162, 0.162, 0.974]
}

# DataFrame e exportação
df_metrics = pd.DataFrame(model_metrics)
df_metrics.to_csv('model_metrics.csv', index=False)


In [109]:
# Monta o dicionário
feature_importance = {
    'Feature': feature_cols,
    'Random_Forest': importances_rf,
    'Gradient_Boosting': importances_gbt,
    'Ridge': df_importance_ridge['Importance_Ridge'].values,  # só a coluna de importâncias
    'Lasso': df_importance_lasso['Importance_Lasso'].values,  # só a coluna de importâncias
    'XGBoost': importances_xgb_full,
}
df_importance = pd.DataFrame(feature_importance)
df_importance.to_csv('feature_importance.csv', index=False)


In [110]:
# Selecionando colunas de interesse
impact_df = df[['Date', 'Weekly_Sales', 'CPI', 'Unemployment', 'Temperature']]
impact_df.to_csv('impact_over_time.csv', index=False)


In [111]:
# Criando um DataFrame para armazenar previsões
predictions_df = pd.DataFrame({
    'Date': df['Date'],
    'Weekly_Sales': df['Weekly_Sales'],
    'Random_Forest': rf_predictions,
    'Gradient_Boosting': gbt_predictions,
    'Ridge': ridge_predictions,
    'Lasso': lasso_predictions,
    'XGBoost': xgb_predictions
})

predictions_df.to_csv('model_predictions.csv', index=False)
